# TensorFlow에서 모델 저장하고 불러오기 - Image classification with MNIST dataset
MNIST image classification 코드 예제를 통해 tf.train.Saver를 이용하여 모델을 저장하고 불러오는 방법을 설명합니다.

### 라이브러리 import(tensorflow 설치필요)및 MNIST 데이터셋 다운로드

In [1]:
import tensorflow as tf
import os
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use urllib or similar directly.
Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from t

### 모델 만들기

In [2]:

def CNN(x):
  x_image = tf.reshape(x, [-1, 28, 28, 1])

  W_conv1 = tf.Variable(tf.truncated_normal(shape=[5, 5, 1, 32], stddev=5e-2))
  b_conv1 = tf.Variable(tf.constant(0.1, shape=[32]))
  h_conv1 = tf.nn.relu(tf.nn.conv2d(x_image, W_conv1, strides=[1, 1, 1, 1], padding='SAME') + b_conv1)
  h_pool1 = tf.nn.max_pool(h_conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

  W_conv2 = tf.Variable(tf.truncated_normal(shape=[5, 5, 32, 64], stddev=5e-2))
  b_conv2 = tf.Variable(tf.constant(0.1, shape=[64]))
  h_conv2 = tf.nn.relu(tf.nn.conv2d(h_pool1, W_conv2, strides=[1, 1, 1, 1], padding='SAME') + b_conv2)
  h_pool2 = tf.nn.max_pool(h_conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

  W_fc1 = tf.Variable(tf.truncated_normal(shape=[7 * 7 * 64, 1024], stddev=5e-2))
  b_fc1 = tf.Variable(tf.constant(0.1, shape=[1024]))
  h_pool2_flat = tf.reshape(h_pool2, [-1, 7*7*64])
  h_fc1 = tf.nn.relu(tf.matmul(h_pool2_flat, W_fc1) + b_fc1)

  W_output = tf.Variable(tf.truncated_normal(shape=[1024, 10], stddev=5e-2))
  b_output = tf.Variable(tf.constant(0.1, shape=[10]))
  logits = tf.matmul(h_fc1, W_output) + b_output
  y_pred = tf.nn.softmax(logits)

  return y_pred, logits

x = tf.placeholder(tf.float32, shape=[None, 784])
y = tf.placeholder(tf.float32, shape=[None, 10])

y_pred, logits = CNN(x)

loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=logits))
train_step = tf.train.AdamOptimizer(1e-4).minimize(loss)

correct_prediction = tf.equal(tf.argmax(y_pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

print("done")


Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

done


### tf.train.Saver 사용하여 모델 저장

In [3]:

SAVER_DIR = "model"

saver = tf.train.Saver()
checkpoint_path = os.path.join(SAVER_DIR, "model")
ckpt = tf.train.get_checkpoint_state(SAVER_DIR)

print("done")

done


### 학습 시작

In [4]:
# start training
with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  for step in range(1000):
    batch = mnist.train.next_batch(50)
    # 100 step마다 tf.train.Saver를 이용해서 모델 파라미터 저장
    if step % 100 == 0:      
      saver.save(sess, checkpoint_path, global_step=step)
      train_accuracy = accuracy.eval(feed_dict={x: batch[0], y: batch[1]})
      print("Epoch: %d, train_accuracy: %f" % (step, train_accuracy))
    sess.run([train_step], feed_dict={x: batch[0], y: batch[1]})

  print("test accuracy: %f" % accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))

Epoch: 0, train_accuracy: 0.060000
Epoch: 100, train_accuracy: 0.800000
Epoch: 200, train_accuracy: 0.880000
Epoch: 300, train_accuracy: 0.940000
Epoch: 400, train_accuracy: 0.900000
Instructions for updating:
Use standard file APIs to delete files with this prefix.
Epoch: 500, train_accuracy: 0.960000
Epoch: 600, train_accuracy: 0.920000
Epoch: 700, train_accuracy: 0.940000
Epoch: 800, train_accuracy: 0.980000
Epoch: 900, train_accuracy: 0.960000
test accuracy: 0.967400


### 훈련되지 않은 새로운 모델 생성하여 테스트셋으로 성능 평가 후 저장된 체크포인트를 로드하여 다시 평가

In [0]:

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())

  print("학습되지 않은 모델 정확도: %f" % accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))
  exit()

with tf.Session() as sess:
  sess.run(tf.global_variables_initializer())
  
  if ckpt and ckpt.model_checkpoint_path:
    saver.restore(sess, ckpt.model_checkpoint_path)    
    print("복원된 모델 정확도 : %f" % accuracy.eval(feed_dict={x: mnist.test.images, y: mnist.test.labels}))
    sess.close()
    exit()




W0830 05:58:59.176173 140273426544512 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


학습되지 않은 모델 정확도: 0.110200
복원된 모델 정확도 : 0.961300
